**Programa Para o Twitter***

In [58]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkTwitterV02") \
    .getOrCreate()

In [59]:
#bibliotecas utilizadas para realizar a análise dos textos
from textblob import TextBlob  #utilizada para realizar o processamento do texto e análise de sentimento
from googletrans import Translator #utilizado para traduzir textos 
from unidecode import unidecode  #utilizada para "decodificar caracteres" não textuais

In [60]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas através do localhost e porta 9999
# define a fonte (source) de dados
twitters = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9995) \
    .load()

23/01/27 01:34:29 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [61]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, FloatType

In [62]:
#função para realizar a tradução para o inglês 
def translate_udf(col):
    trans_obj=Translator().translate(col)
    return trans_obj.text

#função para realizar a análise de sentimento
def sentiment_udf(col):
    sentiment_text=TextBlob(col)
    return sentiment_text.polarity


In [63]:
#definição das função como User-Defined-Function
unicode_udf_string = udf(lambda z: unidecode(z), StringType()) #define a função de decode para ser utilizada no dataframe
group_by_sentiment = udf(lambda x: 'negativo' if x < -0.1 else 'positivo' if x > 0.1 else 'neutro',StringType())
translate_udf_string = udf(translate_udf, StringType()) #define a função de tradução
sentiment_udf_float = udf(sentiment_udf, FloatType()) #define a função de tradução

In [64]:
#testando funções
teste="Eu ♥ o meu cachorro, ele é o meu melhor amigo"
decode=unidecode(teste)
print(decode)
decodeEN=Translator().translate(decode)
print(decodeEN.text)
a=str(decodeEN)
sentiment = TextBlob(a)
print(sentiment.polarity)

Eu  o meu cachorro, ele e o meu melhor amigo
Me my dog, he is my best friend
1.0


In [65]:
#Aplica as funções udf para a seleção de colunas
twitters_unicode=twitters.select( "value",unicode_udf_string(twitters.value).alias("unicoded")) #decodifica
twitters_uni_trans=twitters_unicode.select( "value","unicoded",translate_udf_string(col("unicoded")).alias("twitter_EN")) #traduz
twitters_uni_trans_sent=twitters_uni_trans.select("value", "unicoded", "twitter_EN", sentiment_udf_float(col("twitter_EN")).alias("analise"))#análise de sentimento
t_sent_label= twitters_uni_trans_sent.select("value","unicoded","twitter_EN", "analise",group_by_sentiment(col("analise")).alias("classificacao"))

In [67]:
t_sent_count = t_sent_label.groupBy("classificacao").count() 

In [68]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado 
query = t_sent_count \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

query.awaitTermination() #aguarda até que a "streaming query" termine 

23/01/27 01:34:30 ERROR MicroBatchExecution: Query [id = 537b7b47-9b0a-43af-b4c4-77b3fd30f970, runId = 35dad2f3-e20d-471b-ab4d-4f7a0fb5a272] terminated with error
java.net.ConnectException: Connection refused
	at java.base/sun.nio.ch.Net.connect0(Native Method)
	at java.base/sun.nio.ch.Net.connect(Net.java:579)
	at java.base/sun.nio.ch.Net.connect(Net.java:568)
	at java.base/sun.nio.ch.NioSocketImpl.connect(NioSocketImpl.java:585)
	at java.base/java.net.SocksSocketImpl.connect(SocksSocketImpl.java:327)
	at java.base/java.net.Socket.connect(Socket.java:633)
	at java.base/java.net.Socket.connect(Socket.java:583)
	at java.base/java.net.Socket.<init>(Socket.java:507)
	at java.base/java.net.Socket.<init>(Socket.java:287)
	at org.apache.spark.sql.execution.streaming.sources.TextSocketMicroBatchStream.initialize(TextSocketMicroBatchStream.scala:71)
	at org.apache.spark.sql.execution.streaming.sources.TextSocketMicroBatchStream.planInputPartitions(TextSocketMicroBatchStream.scala:117)
	at org.

StreamingQueryException: Query [id = 30d82275-bc97-4894-904c-81ca4119d782, runId = 127aa2a6-def8-4213-8efc-7fe4d0b82adf] terminated with exception: Connection refused